# Non-intrusive Polynomial Chaos
Suppose that $X$ is a real-valued Gaussian random variable with mean $\mu \in \mathbb{R}$ and standard deviation $\sigma \geq 0$.
We are interested in the PCE coefficients of the random variable $Y = \exp(X)$.

## Analytical Solution
We know that $Y$ is a [log-normal random variable](https://en.wikipedia.org/wiki/Log-normal_distribution).
Thanks to [this book](https://link.springer.com/book/10.1007/978-3-319-23395-6) we know even more: the analytical solution of the PCE coefficients $Y = \sum_{k \in \mathcal{K}} y_k \phi_k$:
$$y_k = \frac{\mathrm{e}^{\mu + \sigma^2/2} \sigma^k}{k!}.$$

Let's code this reference solution.

In [1]:
μ, σ = 1, 0.1
k_values = 0:10
function computeCoefficient_analytic(k::Int)
    exp(μ + 0.5*σ^2) * σ^k / factorial(k)
end

y_ana = computeCoefficient_analytic.(k_values)

11-element Array{Float64,1}:
 2.731907272825927     
 0.2731907272825927    
 0.013659536364129636  
 0.00045531787880432126
 1.138294697010803e-5  
 2.276589394021606e-7  
 3.794315656702678e-9  
 5.4204509381466814e-11
 6.775563672683354e-13 
 7.528404080759281e-15 
 7.528404080759283e-17 

## Numerical Solution
We now want to construct the solution numerically using `PolyChaos`.
The definition of the $k$-th PCE coefficient is
$$
y_k = \frac{\langle \mathrm{e}^{\mu + \sigma \Xi}, \phi_k \rangle}{\langle \phi_k, \phi_k \rangle},
$$
where $\Xi$ is a standard Gaussian random variable with mean zero and unit variance.
A few manipulations lead us to the integral
$$
y_k = \frac{\mathrm{e}^{\mu + \sigma^2/2}}{k! \sqrt{2 \pi}} \int_{\mathbb{R}} \mathrm{He}_k(w + \sigma) \mathrm{e}^{- x^2/2} \mathrm{d}w,
$$
which we can solve using `PolyChaos.jl`

In [4]:
using PolyChaos
maxdegree = maximum(k_values)
op = GaussOrthoPoly(maxdegree, Nrec=2*maxdegree)

function computeCoefficients_numeric(k::Int)
    He_k(w) = evaluate(k, w + σ, op)
    exp(μ + 0.5*σ^2) / factorial(k) * integrate(He_k, op)
end

y_num = computeCoefficients_numeric.(k_values)

11-element Array{Float64,1}:
 2.7319072728259264   
 0.27319072728259286  
 0.013659536364131822 
 0.0004553178788047899
 1.1382946970000281e-5
 2.2765893917412756e-7
 3.7943156337759115e-9
 5.420454653270979e-11
 6.775600262027881e-13
 7.524514560809339e-15
 7.502338086662093e-17

Comparing the results we see that the results coincide.

In [5]:
using LinearAlgebra
norm(y_ana - y_num, Inf)

2.185751579730777e-15